Домашнее задание по теме «Ансамблирование»
Преподаватель: Алексей Кузьмин
Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
Построить случайный лес, вывести важность признаков
Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
В качестве решения: Jupyter notebook с кодом, комментариями и графиками, ссылка на гитхаб

In [132]:
# Include external libraries
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

In [133]:
# Load of dataset, printing of shape of dataset
data_train = pd.read_csv('./train.csv')
data_test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
print("train shape ===================================================")
print(data_train.shape)
print("test shape ===================================================")
print(data_test.shape)

train shape ===================================================
(1460, 81)
test shape ===================================================
(1459, 80)


In [134]:
# Observing of train data
data_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [135]:
# VObserving of test data
data_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [136]:
# Observing of sample submission data
sample_submission.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [137]:
# VObserving of sample submission structure
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
Id           1459 non-null int64
SalePrice    1459 non-null float64
dtypes: float64(1), int64(1)
memory usage: 22.9 KB


In [138]:
# Load significant columns (previously manually selected)
columns = pd.read_csv('./columns.csv')
# Convertion to suitable type
columns['Columns'] = columns['Columns'].astype('str')
# Observing of columns structure
print(columns.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 1 columns):
Columns    35 non-null object
dtypes: object(1)
memory usage: 360.0+ bytes
None


In [139]:
# Split of columns by data type
columns_flag = columns.iloc[3:6]
columns_bin = columns.iloc[7:9]
columns_category = columns.iloc[10:20]
columns_date = columns.iloc[21:22]
columns_number = columns.iloc[25:35]

In [140]:
# Aligning of row number
data_train.drop(data_train.index[[1459]], inplace=True)
# Concatenattion of train dataset columns
data_usefull_tr = data_train[np.concatenate((columns_flag['Columns'].values, \
                                             columns_bin['Columns'].values, \
                                             columns_category['Columns'].values, \
                                             columns_date['Columns'].values, \
                                             columns_number['Columns'].values))]
# Concatenattion of test dataset columns
data_usefull_te = data_test[np.concatenate((columns_flag['Columns'].values, \
                                            columns_bin['Columns'].values, \
                                            columns_category['Columns'].values, \
                                            columns_date['Columns'].values, \
                                            columns_number['Columns'].values))]
# Extraction of target data
Y_train = data_train[['SalePrice']]

In [141]:
# Observing of train dataset structure
data_usefull_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 26 columns):
LandContour     1459 non-null object
Foundation      1459 non-null object
HouseStyle      1459 non-null object
Street          1459 non-null object
Alley           91 non-null object
Utilities       1459 non-null object
ExterQual       1459 non-null object
ExterCond       1459 non-null object
BsmtFinType1    1422 non-null object
HeatingQC       1459 non-null object
KitchenQual     1459 non-null object
GarageFinish    1378 non-null object
PavedDrive      1459 non-null object
PoolQC          7 non-null object
Fence           281 non-null object
YrSold          1459 non-null int64
OverallQual     1459 non-null int64
OverallCond     1459 non-null int64
GarageCars      1459 non-null int64
GarageArea      1459 non-null int64
LotArea         1459 non-null int64
BsmtFinSF2      1459 non-null int64
BsmtUnfSF       1459 non-null int64
TotalBsmtSF     1459 non-null int64
TotRmsAbvGrd    1459

In [142]:
# Observing of test dataset structure and compater with train dataset structure
data_usefull_te.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 26 columns):
LandContour     1459 non-null object
Foundation      1459 non-null object
HouseStyle      1459 non-null object
Street          1459 non-null object
Alley           107 non-null object
Utilities       1457 non-null object
ExterQual       1459 non-null object
ExterCond       1459 non-null object
BsmtFinType1    1417 non-null object
HeatingQC       1459 non-null object
KitchenQual     1458 non-null object
GarageFinish    1381 non-null object
PavedDrive      1459 non-null object
PoolQC          3 non-null object
Fence           290 non-null object
YrSold          1459 non-null int64
OverallQual     1459 non-null int64
OverallCond     1459 non-null int64
GarageCars      1458 non-null float64
GarageArea      1458 non-null float64
LotArea         1459 non-null int64
BsmtFinSF2      1458 non-null float64
BsmtUnfSF       1458 non-null float64
TotalBsmtSF     1458 non-null float64
TotRmsAbv

In [143]:
# Observing of target data structure
Y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 1 columns):
SalePrice    1459 non-null int64
dtypes: int64(1)
memory usage: 22.8 KB


In [144]:
# Filling of not available cells for train dataset
data_usefull_tr['Alley'].fillna("NA", inplace=True)
data_usefull_tr['GarageFinish'].fillna("NA", inplace=True)
data_usefull_tr['PoolQC'].fillna("NA", inplace=True)
data_usefull_tr['Fence'].fillna("NA", inplace=True)
data_usefull_tr['BsmtFinType1'].fillna("NA", inplace=True)
data_usefull_tr['GarageCars'].fillna(0, inplace=True)
data_usefull_tr['GarageArea'].fillna(0, inplace=True)

/home/user/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [145]:
# Filling of not available cells for test dataset
data_usefull_te['Alley'].fillna("NA", inplace=True)
data_usefull_te['GarageFinish'].fillna("NA", inplace=True)
data_usefull_te['PoolQC'].fillna("NA", inplace=True)
data_usefull_te['Fence'].fillna("NA", inplace=True)
data_usefull_te['BsmtFinType1'].fillna("NA", inplace=True)
data_usefull_te['Utilities'].fillna("ELO", inplace=True)
data_usefull_te['KitchenQual'].fillna("NA", inplace=True)
data_usefull_te['BsmtFinSF2'].fillna(0, inplace=True)
data_usefull_te['BsmtUnfSF'].fillna(0, inplace=True)
data_usefull_te['TotalBsmtSF'].fillna(0, inplace=True)
data_usefull_te['GarageCars'].fillna(0, inplace=True)
data_usefull_te['GarageArea'].fillna(0, inplace=True)

In [146]:
# Test of test dataset non-null number of items
data_usefull_te.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 26 columns):
LandContour     1459 non-null object
Foundation      1459 non-null object
HouseStyle      1459 non-null object
Street          1459 non-null object
Alley           1459 non-null object
Utilities       1459 non-null object
ExterQual       1459 non-null object
ExterCond       1459 non-null object
BsmtFinType1    1459 non-null object
HeatingQC       1459 non-null object
KitchenQual     1459 non-null object
GarageFinish    1459 non-null object
PavedDrive      1459 non-null object
PoolQC          1459 non-null object
Fence           1459 non-null object
YrSold          1459 non-null int64
OverallQual     1459 non-null int64
OverallCond     1459 non-null int64
GarageCars      1459 non-null float64
GarageArea      1459 non-null float64
LotArea         1459 non-null int64
BsmtFinSF2      1459 non-null float64
BsmtUnfSF       1459 non-null float64
TotalBsmtSF     1459 non-null float64
TotR

In [147]:
# Train and test datasets transformation for  prepare to learning

# Get dummies for flag features
data_flag_tr = data_usefull_tr[columns_flag['Columns'].values]
data_flag_tr = pd.get_dummies(data_flag_tr)

data_flag_te = data_usefull_te[columns_flag['Columns'].values]
data_flag_te = pd.get_dummies(data_flag_te)

# Get dummies for binary features (or considered as binary)
data_bin_tr = data_usefull_tr[columns_bin['Columns'].values]
data_bin_tr['Street'] = np.where(data_bin_tr['Street'].str.contains("Pave"), 1, 0)
data_bin_tr['Alley'] = np.where(data_bin_tr['Alley'].str.contains("NA"), 0, 1)

data_bin_te = data_usefull_te[columns_bin['Columns'].values]
data_bin_te['Street'] = np.where(data_bin_te['Street'].str.contains("Pave"), 1, 0)
data_bin_te['Alley'] = np.where(data_bin_te['Alley'].str.contains("NA"), 0, 1)

# Get countable presentation of comparing categorial features
data_category_tr = data_usefull_tr[columns_category['Columns'].values]
data_category_te = data_usefull_te[columns_category['Columns'].values]

cleanup_Utilities = {"Utilities": {"ELO": 0, "NoSeWa":1, "NoSewr":2, "AllPub":3}}
cleanup_ExterQual = {"ExterQual":{"Ex":4, "Gd":3, "TA":2, "Fa":1, "Po":0}}
cleanup_ExterCond = {"ExterCond":{"Ex":4, "Gd":3, "TA":2, "Fa":1, "Po":0}}
cleanup_BsmtFinType1 = {"BsmtFinType1":{"GLQ":6, "ALQ":5, "BLQ":4, "Rec":3, "LwQ":2, "Unf":1, "NA":0}}

cleanup_HeatingQC = {"HeatingQC":{"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "NA": 0}}
cleanup_KitchenQual = {"KitchenQual":{"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "NA": 0}}
cleanup_GarageFinish = {"GarageFinish":{"Fin":3, "RFn":2, "Unf":1, "NA":0}}
cleanup_PavedDrive = {"PavedDrive":{"Y":2, "P":1, "N":0}}
cleanup_PoolQC = {"PoolQC":{"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "NA": 0, "NaN": 0}}
cleanup_Fence = {"Fence":{"GdPrv":4, "MnPrv":3, "GdWo":2, "MnWw":1, "NA":0}}

data_category_tr.replace(cleanup_Utilities, inplace = True)
data_category_tr.replace(cleanup_ExterQual, inplace = True)
data_category_tr.replace(cleanup_ExterCond, inplace = True)
data_category_tr.replace(cleanup_BsmtFinType1, inplace = True)
data_category_tr.replace(cleanup_HeatingQC, inplace = True)
data_category_tr.replace(cleanup_KitchenQual, inplace = True)
data_category_tr.replace(cleanup_GarageFinish, inplace = True)
data_category_tr.replace(cleanup_PavedDrive, inplace = True)
data_category_tr.replace(cleanup_PoolQC, inplace = True)
data_category_tr.replace(cleanup_Fence, inplace = True)

data_category_te.replace(cleanup_Utilities, inplace = True)
data_category_te.replace(cleanup_ExterQual, inplace = True)
data_category_te.replace(cleanup_ExterCond, inplace = True)
data_category_te.replace(cleanup_BsmtFinType1, inplace = True)
data_category_te.replace(cleanup_HeatingQC, inplace = True)
data_category_te.replace(cleanup_KitchenQual, inplace = True)
data_category_te.replace(cleanup_GarageFinish, inplace = True)
data_category_te.replace(cleanup_PavedDrive, inplace = True)
data_category_te.replace(cleanup_PoolQC, inplace = True)
data_category_te.replace(cleanup_Fence, inplace = True)

# Get normalized features for countale feature 
data_date_tr = data_usefull_tr[columns_date['Columns'].values]
data_date_te = data_usefull_te[columns_date['Columns'].values]
sc1 = StandardScaler()
data_date_tr[['YrSold']] = sc1.fit_transform(data_date_tr[['YrSold']] )
# This features were excludes for the best score
#'YearRemodAdd',  'YearBuilt'
data_number_tr = data_usefull_tr[columns_number['Columns'].values]
data_number_tr = data_number_tr.astype('float')
data_number_tr[['OverallQual', 'OverallCond', 'GarageCars', 'GarageArea', 'LotArea', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'Fireplaces']] = sc1.fit_transform(data_number_tr[['OverallQual', 'OverallCond', 'GarageCars', 'GarageArea', 'LotArea', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'Fireplaces']] )

data_date_te[['YrSold']] = sc1.fit_transform(data_date_te[['YrSold']] )
data_number_te = data_usefull_te[columns_number['Columns'].values]
data_number_te = data_number_te.astype('float')
data_number_te[['OverallQual', 'OverallCond', 'GarageCars', 'GarageArea', 'LotArea', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'Fireplaces']] = sc1.fit_transform(data_number_te[['OverallQual', 'OverallCond', 'GarageCars', 'GarageArea', 'LotArea', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'Fireplaces']] )


/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [148]:
# Concatenation of particular datasets
data_usefull_tr.drop(data_usefull_tr.index, inplace=True)
data_usefull_te.drop(data_usefull_te.index, inplace=True)

data_usefull_tr = pd.concat([data_flag_tr, data_bin_tr, data_category_tr, data_date_tr, data_number_tr],axis=1)
data_usefull_te = pd.concat([data_flag_tr, data_bin_te,  data_category_te, data_date_te, data_number_te],axis=1)

/home/user/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [149]:
# Observing of train dataset after "clearing"
data_usefull_tr.head()

,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,...,OverallQual,OverallCond,GarageCars,GarageArea,LotArea,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,TotRmsAbvGrd,Fireplaces
0,0,0,0,1,0,0,1,0,0,0,...,0.650852,-0.516787,0.311027,0.350351,-0.207111,-0.287744,-0.945245,-0.458867,0.911711,-0.951848
1,0,0,0,1,0,1,0,0,0,0,...,-0.072372,2.179252,0.311027,-0.061360,-0.091895,-0.287744,-0.641887,0.466648,-0.318804,0.599824
2,0,0,0,1,0,0,1,0,0,0,...,0.650852,-0.516787,0.311027,0.631062,0.073415,-0.287744,-0.302307,-0.312973,-0.318804,0.599824
3,0,0,0,1,1,0,0,0,0,0,...,0.650852,-0.516787,1.649634,0.790132,-0.096904,-0.287744,-0.062338,-0.686827,0.296454,0.599824
4,0,0,0,1,0,0,1,0,0,0,...,1.374077,-0.516787,1.649634,1.697766,0.374980,-0.287744,-0.175531,0.199936,1.526968,0.599824


In [150]:
# Observing of test dataset after "clearing"
data_usefull_te.head()

,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,...,OverallQual,OverallCond,GarageCars,GarageArea,LotArea,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,TotRmsAbvGrd,Fireplaces
0,0,0,0,1,0,0,1,0,0,0,...,-0.751101,0.400766,-0.984704,1.185511,0.363929,0.517537,-0.649392,-0.368484,-0.918335,-0.898055
1,0,0,0,1,0,1,0,0,0,0,...,-0.054877,0.400766,-0.984704,-0.738518,0.897861,-0.297689,-0.338322,0.639542,-0.255371,-0.898055
2,0,0,0,1,0,0,1,0,0,0,...,-0.751101,-0.497418,0.302646,0.043982,0.809646,-0.297689,-0.953599,-0.264750,-0.255371,0.647066
3,0,0,0,1,1,0,0,0,0,0,...,-0.054877,0.400766,0.302646,-0.011253,0.032064,-0.297689,-0.525879,-0.269260,0.407593,0.647066
4,0,0,0,1,0,0,1,0,0,0,...,1.337571,-0.497418,0.302646,0.154452,-0.971808,-0.297689,1.059203,0.529042,-0.918335,-0.898055


In [151]:
# Cration of regression model objects
regr_dt = DecisionTreeRegressor(max_depth=5, random_state=1)
regr_knn = KNeighborsRegressor()
regr_lr = LinearRegression()
regr_rf = RandomForestRegressor(max_depth=5, random_state=0,  n_estimators=50)

In [152]:
# Final observing and validation of test dataset structure
data_usefull_te.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 41 columns):
LandContour_Bnk      1459 non-null uint8
LandContour_HLS      1459 non-null uint8
LandContour_Low      1459 non-null uint8
LandContour_Lvl      1459 non-null uint8
Foundation_BrkTil    1459 non-null uint8
Foundation_CBlock    1459 non-null uint8
Foundation_PConc     1459 non-null uint8
Foundation_Slab      1459 non-null uint8
Foundation_Stone     1459 non-null uint8
Foundation_Wood      1459 non-null uint8
HouseStyle_1.5Fin    1459 non-null uint8
HouseStyle_1.5Unf    1459 non-null uint8
HouseStyle_1Story    1459 non-null uint8
HouseStyle_2.5Fin    1459 non-null uint8
HouseStyle_2.5Unf    1459 non-null uint8
HouseStyle_2Story    1459 non-null uint8
HouseStyle_SFoyer    1459 non-null uint8
HouseStyle_SLvl      1459 non-null uint8
Street               1459 non-null int64
Alley                1459 non-null int64
Utilities            1459 non-null int64
ExterQual            1459 non-nu

In [153]:
# Final observing and validation of test dataset structure
data_usefull_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 41 columns):
LandContour_Bnk      1459 non-null uint8
LandContour_HLS      1459 non-null uint8
LandContour_Low      1459 non-null uint8
LandContour_Lvl      1459 non-null uint8
Foundation_BrkTil    1459 non-null uint8
Foundation_CBlock    1459 non-null uint8
Foundation_PConc     1459 non-null uint8
Foundation_Slab      1459 non-null uint8
Foundation_Stone     1459 non-null uint8
Foundation_Wood      1459 non-null uint8
HouseStyle_1.5Fin    1459 non-null uint8
HouseStyle_1.5Unf    1459 non-null uint8
HouseStyle_1Story    1459 non-null uint8
HouseStyle_2.5Fin    1459 non-null uint8
HouseStyle_2.5Unf    1459 non-null uint8
HouseStyle_2Story    1459 non-null uint8
HouseStyle_SFoyer    1459 non-null uint8
HouseStyle_SLvl      1459 non-null uint8
Street               1459 non-null int64
Alley                1459 non-null int64
Utilities            1459 non-null int64
ExterQual            1459 non-nu

In [154]:
# Learning of models
regr_dt.fit(data_usefull_tr, Y_train)
dtrY_pred = regr_dt.predict(data_usefull_te)
regr_knn.fit(data_usefull_tr, Y_train)
knnY_pred = regr_knn.predict(data_usefull_te)
regr_lr.fit(data_usefull_tr, Y_train)
lrY_pred = regr_lr.predict(data_usefull_te)
regr_rf.fit(data_usefull_tr, Y_train) 
rfY_pred = regr_rf.predict(data_usefull_te) 

/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [155]:
print("Mean square error for decision tree regressor")
print(mean_absolute_error(Y_train, dtrY_pred))
print("Determination coefficient(score) for decision tree regressor")
print(regr_dt.score(data_usefull_te, dtrY_pred ))
print("Mean square error for KNN regressor")
print(mean_absolute_error(knnY_pred,Y_train))
print("Determination coefficient(score) for KNN regressor")
print(regr_knn.score(data_usefull_te, knnY_pred))
print("Mean square error for linear regressor")
print(mean_absolute_error(lrY_pred,Y_train))
print("Determination coefficient(score) for linear regressor")
print(regr_lr.score(data_usefull_te, lrY_pred))
print("Mean square error for random forest regressor")
print(mean_absolute_error(rfY_pred,Y_train))
print("Determination coefficient(score) for random forest regressor")
print(regr_rf.score(data_usefull_te, rfY_pred))
print("")
print("Random forest feature importance")
print(regr_rf.feature_importances_)

Mean square error for decision tree regressor
78081.78234516353
Determination coefficient(score) for decision tree regressor
1.0
Mean square error for KNN regressor
75805.52981494174
Determination coefficient(score) for KNN regressor
1.0
Mean square error for linear regressor
76488.41809458533
Determination coefficient(score) for linear regressor
1.0
Mean square error for random forest regressor
77746.42233215118
Determination coefficient(score) for random forest regressor
1.0

Random forest feature importance
[3.49004651e-03 3.10307019e-04 9.09425583e-05 1.63967276e-03
 0.00000000e+00 8.63816281e-06 1.83246866e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 6.26677601e-05 0.00000000e+00
 5.07828096e-04 4.09905040e-05 4.96010784e-05 1.12142123e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.18267663e-03
 0.00000000e+00 3.43648971e-03 1.16557838e-03 2.50493045e-03
 4.20171250e-04 2.07778563e-03 8.18853401e-03 7.12841458e-05
 5.08503603e-03 1.80987348e-03 2.73593272e-04 7.20232574e

Stacking

In [156]:
# Usefull function from webinar practice (a little bit changed)
def get_meta_features(reg, X_train, y_train, X_test, stack_cv):
    meta_train = np.zeros_like(y_train, dtype=float)
    meta_test = np.zeros_like(X_test.values[:,0], dtype=float)
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        
        reg.fit(X_train.iloc[train_ind], y_train[train_ind])
        meta_train[test_ind] = reg.predict(X_train.iloc[test_ind])
        meta_test += reg.predict(X_test)
    return meta_train, meta_test / stack_cv.n_splits

In [157]:
#Creation of model objects of first level of stacking
reg_dtr = DecisionTreeRegressor(max_depth=5, random_state=1)
reg_knnr = KNeighborsRegressor()
reg_lr = LinearRegression()
reg_rf = RandomForestRegressor(max_depth=3, random_state=0,  n_estimators=100)
reg_knn = KNeighborsRegressor()
reg_svr = SVR(gamma='scale', C=1.0, epsilon=0.2)
print(Y_train.shape)

(1459, 1)


In [158]:
# Creation of Crossvalidation engine object
stack_cv = StratifiedKFold(n_splits=10, random_state=555)

# Definition of global variables
meta_train = []
meta_test = []
col_names = []

# Get metafeature of linear regressor
print('LR features...')
meta_tr, meta_te = get_meta_features(reg_lr, data_usefull_tr, Y_train.values[:,0], data_usefull_te, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('lr_pred')

# Get metafeature of KNN regressor
print('KNN features...')
meta_tr, meta_te = get_meta_features(reg_knn, data_usefull_tr, Y_train.values[:,0], data_usefull_te, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('knn_pred')

# Get metafeature of Support Vectore Machine regressor
print('SVR features...')
meta_tr, meta_te = get_meta_features(reg_svr, data_usefull_tr, Y_train.values[:,0], data_usefull_te, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('svr_pred')

LR features...


/home/user/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


KNN features...


/home/user/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


SVR features...


/home/user/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [159]:
# Transformation of sequential set of array to set of columns of dataframe
X_meta_train = pd.DataFrame(np.stack(meta_train, axis=1), columns=col_names)
X_meta_test = pd.DataFrame(np.stack(meta_test, axis=1), columns=col_names)

In [160]:
# Cretion of model object of second level of stacking
reg_meta_rf = RandomForestRegressor(max_depth=3, random_state=0,  n_estimators=100)
# Learning of model of second level of stacking
reg_meta_rf.fit(X_meta_train, Y_train)

/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [161]:
    def feature_importances_(self):
        """Return the feature importances (the higher, the more important the
           feature).
        Returns
        -------
        feature_importances_ : array, shape = [n_features]
        """
        if self.estimators_ is None or len(self.estimators_) == 0:
            raise NotFittedError("Estimator not fitted, "
                                 "call `fit` before `feature_importances_`.")

        all_importances = Parallel(n_jobs=self.n_jobs,
                                   backend="threading")(
            delayed(getattr)(tree, 'feature_importances_')
            for tree in self.estimators_)

        return sum(all_importances) / len(self.estimators_)
    
y_pred_meta_test = reg_meta_rf.predict(X_meta_test)
print("Determination coefficient(score) for stacking")
print(reg_meta_rf.score(X_meta_test, y_pred_meta_test))
print("Mean square error for stacking")
print(mean_absolute_error(y_pred_meta_test,Y_train))
print("")
print("Meta feature importance")
print(reg_meta_rf.feature_importances_)

Determination coefficient(score) for stacking
1.0
Mean square error for stacking
74049.44477197532

Meta feature importance
[0.87411619 0.08441813 0.04146568]


In [162]:
#reg_meta_rf.coef_[0]
#coefs = np.abs(reg_meta_rf.coef_[0])
#indices = np.argsort(coefs)[::-1]


In [163]:
#plt.figure(figsize=(14, 6), dpi=80)
#plt.title("Feature importances (Logistic Regression)")
#plt.bar(range(len(X_train.columns)), coefs[indices[:len(X_train.columns)]],
#       color="r", align="center")
#plt.xticks(range(len(X_train.columns)), X_train.columns.values, rotation=45, ha='right')
#plt.subplots_adjust(bottom=0.3)